In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from lm_polygraph.estimators import *
from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.utils.dataset import Dataset
from lm_polygraph.utils.processor import Logger
from lm_polygraph.utils.manager import UEManager
from lm_polygraph.ue_metrics import PredictionRejectionArea
from lm_polygraph.generation_metrics import RougeMetric, BartScoreSeqMetric, ModelScoreSeqMetric, ModelScoreTokenwiseMetric

# Specify HyperParameters

In [ ]:
model_path = "facebook/bart-large-cnn"
device = "cpu"
dataset_name = "xsum"
batch_size = 4
seed = 42

# Initialize Model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = WhiteboxModel(base_model, tokenizer)

# Train and Eval Datasets

In [ ]:
dataset = Dataset.load(
    dataset_name,
    'document', 'summary',
    batch_size=batch_size,
    split="test"
)
dataset.subsample(16, seed=seed)

train_dataset = Dataset.load(
    dataset_name,
    'document', 'summary',
    batch_size=batch_size,
    split="train"
)
train_dataset.subsample(16, seed=seed)

# Metric, UE Metric, and UE Methods

In [ ]:
ue_methods = [MaximumSequenceProbability(), 
              SemanticEntropy(),
              MahalanobisDistanceSeq("decoder"),]

ue_metrics = [PredictionRejectionArea()]

metrics = [RougeMetric('rougeL')]

loggers = [Logger()] 

# Initialize UE Manager

In [ ]:
man = UEManager(
    dataset,
    model,
    ue_methods,
    metrics,
    ue_metrics,
    loggers,
    train_data=train_dataset,
)

# Compute Results

In [ ]:
results = man()

In [ ]:
for key in results.keys():
    print(f"UE Score: {key[1]}, Metric: {key[2]}, UE Metric: {key[3]}, Score: {results[key]:.3f}")